In [1]:
# ensure classes imported from .py files are dynamically updated
%load_ext autoreload
%autoreload 2

# plot matplots nicely
%matplotlib inline  

In [37]:
import numpy as np
import pandas as pd
import os
from climate_drought import config, drought_indices as dri, utils

In [24]:
# configure inputs
aa = config.AnalysisArgs(
    latitude=52.5,
    longitude=1.25,
    start_date='20200101',
    end_date='20221231',
    product='SMA'
)
cf = config.Config(outdir= 'output',indir='input',verbose=False)
cdi = dri.CDI(cf,aa)

In [41]:
cdi.process()

2023-04-05  11:56:18 INFO Initiating processing of ERA5 daily data.
2023-04-05  11:56:19 INFO Input precipitation, 456 values: 0.000 0.005 
2023-04-05  11:56:19 INFO 

2023-04-05  11:56:19 DEBUG scaled values: 0.002 0.012
2023-04-05  11:56:19 DEBUG alphas: 9.687 17.504 betas: 0.000 0.001
2023-04-05  11:56:19 INFO SPI, 456 values: -3.046 2.676
2023-04-05  11:56:21 INFO Completed processing of ERA5 soil water data.
2023-04-05  11:56:24 INFO Completed processing of ERA5 fAPAR data.
2023-04-05  11:56:24 INFO Computing Combined Drought Indicator...
2023-04-05  11:56:24 INFO Completed processing of ERA5 CDI data.


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[       nan        nan        nan 0.9281408  0.9281408  0.9281408
 1.10604626 1.10604626 1.10604626 0.26510123]
[  nan   nan  0.54  0.19 -0.08 -0.05 -0.14 -0.05  0.15  0.  ]
[   nan    nan    nan -1.371 -1.078 -0.787 -1.231 -0.999 -0.859 -0.076]
DatetimeIndex(['2020-01-01', '2020-01-11', '2020-01-21', '2020-02-01',
               '2020-02-11', '2020-02-21', '2020-03-01', '2020-03-11',
               '2020-03-21', '2020-04-01'],
              dtype='datetime64[ns]', name='time', freq=None)
Sam:  {'SPI': None, 'SMA': None, 'fAPAR': None, 'CDI': 0.0}
Sam:  {'SPI': None, 'SMA': None, 'fAPAR': None, 'CDI': 0.0}
Sam:  {'SPI': None, 'SMA': 0.5400000215, 'fAPAR': None, 'CDI': 0.0}
Sam:  {'SPI': 0.9281408024, 'SMA': 0.1899999976, 'fAPAR': -1.3710000515, 'CDI': 0.0}
Sam:  {'SPI': 0.9281408024, 'SMA': -0.0799999982, 'fAPAR': -1.0779999495, 'CDI': 0.0}
Sam:  {'SPI': 0.9281408024, 'SMA': -0.0500000007, 'fAPAR': -0.7870000005, 'CDI': 0.0}
Sam:  {'SPI': 1.1060462582, 'S

,SPI,SMA,fAPAR,CDI
time,,,,
2020-01-01,NaN,NaN,NaN,0
2020-01-11,NaN,NaN,NaN,0
2020-01-21,NaN,0.54,NaN,0
2020-02-01,0.928141,0.19,-1.371,0
2020-02-11,0.928141,-0.08,-1.078,0
2020-02-21,0.928141,-0.05,-0.787,0
2020-03-01,1.106046,-0.14,-1.231,0
2020-03-11,1.106046,-0.05,-0.999,0
2020-03-21,1.106046,0.15,-0.859,0


In [34]:
idx_np = lambda df: df.to_numpy().flatten()
shift_date = lambda arr, n: np.pad(arr[:-n],(n,0),'constant',constant_values=(np.nan,))

# use fAPAR times as CDI delivery dates
time = df_fpr.index

# fapar needs to use previous dekad's values
fpr = shift_date(idx_np(df_fpr),1)

# sma needs to use dekad -2 values
sma = shift_date(idx_np(df_sma),2)

# convert spi to dekads
spi = idx_np(df_spi.spi).repeat(3)[:-2]

# spi needs to be from the previous month
spi = shift_date(spi,3)

# spi is in months, and others are in dekads, so repeat spi 3 times before combining with others
#spi = spi.repeat(3)


In [40]:
time

DatetimeIndex(['2020-01-01', '2020-01-11', '2020-01-21', '2020-02-01',
               '2020-02-11', '2020-02-21', '2020-03-01', '2020-03-11',
               '2020-03-21', '2020-04-01'],
              dtype='datetime64[ns]', name='time', freq=None)

In [39]:
df_spi.inerp

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [38]:
utils.to_dekads(df_spi)

,tp,spi
2020-01-01,0.001882,0.928141
2020-02-01,0.002690,1.106046
2020-03-01,0.000918,0.265101
2020-04-01,0.001049,-0.339632


,tp,spi
time,,
2020-01-01,0.001882,0.928141
2020-02-01,0.002690,1.106046
2020-03-01,0.000918,0.265101
2020-04-01,0.001049,-0.339632


In [16]:
arr=np.pad(df.to_numpy().flatten()[:-1],(1,0),'constant',constant_values=(np.nan,))

In [22]:
time=df.index

In [35]:
pd.DataFrame({'sdf':arr},index=time)

,sdf
time,
2020-01-01,NaN
2020-01-11,NaN
2020-01-21,NaN
2020-02-01,-1.371
2020-02-11,-1.078
2020-02-21,-0.787
2020-03-01,-1.231
2020-03-11,-0.999
2020-03-21,-0.859
